GWAS loci are taken from the CLOZUK paper Pardiñas et al Nature genetics, 2018, 50, 381-389  DOI 10.1038/s41588-018-0059-2.  This paper presented a meta analysis of the author's own GWAS and the earlier PGC2 GWAS.  CLOZUK comes from CLOZapine UK; clozapine is an antipsychotic medication mainly used for schizophrenia.

In [19]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import pandas as pd
import numpy as np
import genome
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
fpath = '/home/attila/projects/bsm/resources/CLOZUK/supp-table-4.csv'
gwas = pd.read_csv(fpath, skiprows=7)
gwas['Chromosome'] = pd.Categorical(gwas['Chromosome'], ordered=True, categories=[str(y) for y in range(1, 23)] + list('XY'))
gwas.sort_values(['Chromosome', 'Start (BP)'], inplace=True)
gwas

,Locus,GWS in PGC2?,Chromosome,Start (BP),End (BP),Index SNP (dbSNP b141),A1,A2,OR,SE,P-value,Number of SNPs,Length (KB),Gene(s) tagged
40,41,Yes,1,2372397,2402499,rs4648845,C,T,0.926,0.011,6.740000e-12,27,30.102,PLCH2
86,87,Yes,1,8392592,8701288,rs34269918,G,GA,0.941,0.010,3.270000e-09,56,308.696,"RERE, SLC45A1"
38,39,Yes,1,30428943,30459412,rs6694545,A,G,1.080,0.011,6.200000e-12,46,30.469,LOC101929406
45,46,Yes,1,44029353,44137257,rs2970610,T,C,1.070,0.010,1.390000e-11,73,107.904,"KDM4A, PTPRF"
135,136,No,1,66304167,66333877,rs12129719,G,A,0.947,0.010,3.350000e-08,23,29.710,PDE4B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27,28,Yes,22,39840130,40091818,rs5757730,A,G,0.931,0.010,1.760000e-12,49,251.688,"LOC100506472, MGAT3, TAB1, CACNA1I, RPS19BP1"
24,25,Yes,22,41408754,41684093,rs9607782,T,A,0.922,0.011,5.540000e-13,104,275.339,"CHADL, EP300, EP300-AS1, L3MBTL2, MIR1281, MIR..."
13,14,Yes,22,42315790,42689370,rs6002655,C,T,0.928,0.010,2.150000e-14,119,373.580,"CENPM, LINC00634, MIR33A, SEPT3, SHISA8, SREBF..."
60,61,Yes,X,5859733,6029533,rs12009217,A,G,0.937,0.010,1.780000e-10,51,169.801,NLGN4X


In [3]:
%%bash
cd /home/attila/projects/bsm/resources/CLOZUK
fai=/big/data/refgenome/GRCh37/dna/hs37d5.fa.fai
# The genome.py module uses the bed file below
bed=/big/data/refgenome/GRCh37/dna/hs37d5.bed
sed 's/^\(\S\+\t\)\(\S\+\)\t.*$/\10\t\2/; /^Y/ q' $fai > $bed

In [8]:
coord_cols = ['Chromosome', 'Start (BP)', 'End (BP)']
cgwas = genome.complement_intervals(gwas, coord_cols=coord_cols, onebased=True) # complemented gwas data frame

In [21]:
cgwas['-log p-value'] = - np.log10(cgwas.loc[:, 'P-value'])
cgwas

,Locus,GWS in PGC2?,Chromosome,Start (BP),End (BP),Index SNP (dbSNP b141),A1,A2,OR,SE,P-value,Number of SNPs,Length (KB),Gene(s) tagged,-log p-value
0,NaN,NaN,1,0,2372395,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,41.0,Yes,1,2372395,2402499,rs4648845,C,T,0.926,0.011,6.740000e-12,27.0,30.102,PLCH2,11.171340
1,NaN,NaN,1,2402499,8392590,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86,87.0,Yes,1,8392590,8701288,rs34269918,G,GA,0.941,0.010,3.270000e-09,56.0,308.696,"RERE, SLC45A1",8.485452
2,NaN,NaN,1,8701288,30428941,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,61.0,Yes,X,5859731,6029533,rs12009217,A,G,0.937,0.010,1.780000e-10,51.0,169.801,NLGN4X,9.749580
166,NaN,NaN,X,6029533,68377124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,27.0,Yes,X,68377124,68377205,rs62606711,A,G,1.075,0.010,1.260000e-12,4.0,0.079,PJA1,11.899629
167,NaN,NaN,X,68377205,155270560,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
bedpath = '/home/attila/projects/bsm/resources/CLOZUK/gwas.bed'
cgwas[coord_cols + ['-log p-value']].to_csv(bedpath, sep='\t', header=False, index=False, na_rep='0')

In [6]:
%connect_info

{
  "shell_port": 57061,
  "iopub_port": 45739,
  "stdin_port": 44013,
  "control_port": 57027,
  "hb_port": 50257,
  "ip": "127.0.0.1",
  "key": "d42bb3f4-5c7de9aeb97657c416b261eb",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-63646bca-a9cc-4e07-975b-e10d2f8e799d.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.
